In [144]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.linear_model import LassoCV

In [118]:
df=pd.read_csv('Algerian_forest_fires_dataset_UPDATE_cleaned.csv')

In [119]:
df.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
0,1,6,2012,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,not fire
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,not fire
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire
3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,not fire
4,5,6,2012,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,not fire


In [120]:
df.shape

(244, 14)

In [121]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [122]:
y

0      not fire
1      not fire
2      not fire
3      not fire
4      not fire
         ...   
239        fire
240    not fire
241    not fire
242    not fire
243    not fire
Name: Classes, Length: 244, dtype: object

In [123]:
X.corr()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI
day,1.000000e+00,2.232788e-17,NaN,0.095772,-0.074209,0.047001,-0.112265,0.224032,0.491571,0.527930,0.177727,0.517229,0.350756
month,2.232788e-17,1.000000e+00,NaN,-0.059017,-0.037884,-0.041447,0.035322,0.015577,0.068178,0.127669,0.061680,0.085822,0.082613
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Temperature,9.577222e-02,-5.901677e-02,NaN,1.000000,-0.654443,-0.278132,-0.326786,0.677491,0.483105,0.370511,0.607551,0.455504,0.564573
RH,-7.420934e-02,-3.788419e-02,NaN,-0.654443,1.000000,0.236084,0.222968,-0.645658,-0.405133,-0.220344,-0.690637,-0.348587,-0.577544
Ws,4.700086e-02,-4.144673e-02,NaN,-0.278132,0.236084,1.000000,0.170169,-0.163255,-0.001246,0.076253,0.015248,0.029756,0.032298
Rain,-1.122654e-01,3.532207e-02,NaN,-0.326786,0.222968,0.170169,1.000000,-0.544045,-0.288548,-0.296808,-0.347105,-0.299171,-0.324363
FFMC,2.240321e-01,1.557668e-02,NaN,0.677491,-0.645658,-0.163255,-0.544045,1.000000,0.602391,0.503919,0.739730,0.589652,0.690274
DMC,4.915710e-01,6.817778e-02,NaN,0.483105,-0.405133,-0.001246,-0.288548,0.602391,1.000000,0.875362,0.674499,0.982073,0.875830
DC,5.279300e-01,1.276691e-01,NaN,0.370511,-0.220344,0.076253,-0.296808,0.503919,0.875362,1.000000,0.498926,0.941906,0.738728


In [124]:
# Calculate the correlation matrix
corr_matrix = X.corr().abs()

# Create an upper triangle matrix of correlations (to avoid duplication)
upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# Find columns with correlation greater than 0.85
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.85)]

In [125]:
print(to_drop)

['DC', 'BUI', 'FWI']


In [126]:
to_drop = to_drop + ["day", "month", "year"]

In [127]:
# Drop the columns
X_reduced = X.drop(columns=to_drop)

In [128]:
X_reduced.corr()

,Temperature,RH,Ws,Rain,FFMC,DMC,ISI
Temperature,1.000000,-0.654443,-0.278132,-0.326786,0.677491,0.483105,0.607551
RH,-0.654443,1.000000,0.236084,0.222968,-0.645658,-0.405133,-0.690637
Ws,-0.278132,0.236084,1.000000,0.170169,-0.163255,-0.001246,0.015248
Rain,-0.326786,0.222968,0.170169,1.000000,-0.544045,-0.288548,-0.347105
FFMC,0.677491,-0.645658,-0.163255,-0.544045,1.000000,0.602391,0.739730
DMC,0.483105,-0.405133,-0.001246,-0.288548,0.602391,1.000000,0.674499
ISI,0.607551,-0.690637,0.015248,-0.347105,0.739730,0.674499,1.000000


In [129]:
# Convert 'Yes' to 1 and 'No' to 0
y = y.map({'fire': 1, 'not fire': 0})

In [130]:
y.value_counts()

Classes
1    138
0    106
Name: count, dtype: int64

In [131]:
X_train,X_test,y_train,y_test=train_test_split(X_reduced,y,test_size=0.25,random_state=42)

linear

In [132]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [133]:
reg = LinearRegression().fit(X_train, y_train)

In [134]:
y_pred=reg.predict(X_test)

In [135]:
score=r2_score(y_test,y_pred)
print(score)

0.6365694912794921


polynomial

In [136]:
poly=PolynomialFeatures(degree=2,include_bias=True)
X_train_poly=poly.fit_transform(X_train)
X_test_poly=poly.transform(X_test)

In [137]:
reg = LinearRegression().fit(X_train_poly, y_train)

In [138]:
y_pred=reg.predict(X_test_poly)

In [139]:
score=r2_score(y_test,y_pred)
print(score)

0.7512982875071385


lasso

In [140]:
clf = linear_model.Lasso(alpha=0.1)

In [141]:
clf.fit(X_train, y_train)

Lasso(alpha=0.1)

In [142]:
y_pred=clf.predict(X_test)

In [143]:
score=r2_score(y_test,y_pred)
print(score)

0.582372825274776


Lasso cv

In [145]:
reg = LassoCV(cv=5, random_state=0).fit(X_train, y_train)

In [146]:
y_pred=reg.predict(X_test)

In [147]:
score=r2_score(y_test,y_pred)
print(score)

0.6402170727380132


In [149]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
ridge=Ridge()
ridge.fit(X_train,y_train)
y_pred=ridge.predict(X_test)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)
# plt.scatter(y_test,y_pred)

Mean absolute error 0.24653454711437622
R2 Score 0.6383363773839557


In [152]:
from sklearn.linear_model import RidgeCV
ridgecv=RidgeCV(cv=5)
ridgecv.fit(X_train,y_train)
y_pred=ridgecv.predict(X_test)
# plt.scatter(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)

Mean absolute error 0.24653454711437622
R2 Score 0.6383363773839557
